In [1]:
import json
import requests
import pandas as pd
from fake_useragent import UserAgent
from bs4 import BeautifulSoup as bs
import time
from tqdm import tqdm
ua = UserAgent(path='fake_useragent.json')
import re
import datetime

In [2]:
newsdf = pd.read_feather('all_news.txt')
#获取交易日
close = pd.read_feather('BasicFactor_Close.txt').set_index('time')
close.index = pd.to_datetime(close.index.astype(str))
close = close['2022-07-01':]
tradeday = close.index
#交易日股票的交易状态
tradestatus = close.mask(close>0,1)

In [3]:
news_hot = pd.DataFrame()
for tradedate in tqdm(tradeday):
    #选择交易日对应的全部热搜按照最高排名排序
    daynewsdf = newsdf[newsdf['交易日']==tradedate.date()].sort_values('最高排名').head(30)
    news_hot = pd.concat([news_hot,daynewsdf])

  0%|          | 0/243 [00:00<?, ?it/s]

100%|██████████| 243/243 [00:02<00:00, 117.07it/s]


In [4]:
cookielist = ['SUB=_2AkMSUdvAf8NxqwJRmfsWyWrqaIx_zw7EieKkDSobJRMxHRl-yT9kqkw-tRB6OdH1L80idvhOHjmOWHyh0aUOq4dcnkhv',
              'SUB=_2A25IMKGBDeRhGeFJ61sU9ivMyz6IHXVrT7tJrDV8PUNbmtB-LVfdkW9NfL7soISUiY-pBSWmLSFARB6DTkTSv7Bu'
              ]

In [5]:
news_hot=pd.read_feather('all_news_full.txt').set_index('index')

In [6]:
i = 0
news_hot_full = pd.DataFrame()
for news in tqdm(news_hot.iterrows(),total=len(news_hot)):
    news_full = news[1]
    try:
        startday = str(news[1].loc['上榜时间'].date()-datetime.timedelta(days=1))
        endday = str(news[1].loc['上榜时间'].date()+datetime.timedelta(days=1))
        url = 'https://s.weibo.com/weibo?q={}&xsort=hot&suball=1&timescope=custom%3A{}%3A{}&Refer=g'.format(str(news[1].loc['热搜']),startday,endday)
        uastr = ua.random
        cookie = cookielist[i]
        headers = {'User-Agent':uastr,'Cookie':cookie}
        res = requests.get(url=url,headers=headers)
        soup = bs(res.text, "html.parser")
        news_full_str = repr(soup.find_all(class_='txt')[0].text)
        if news_full_str[-5:] == '展开c \'':
            news_full_str = repr(soup.find_all(class_='txt')[1].text)
        #正则删去一些网页转义字符
        news_full_str = re.sub(r"\\u.{2,6}[a-zA-Z0-9]", "",news_full_str)
        news_full_str = news_full_str.replace(" ","") 
        news_full_str = news_full_str.replace("\\n","")
        news_full_str = news_full_str.replace("收起d","")
        news_full.loc['全文_hot'] = news_full_str
    except:
        #防止爬虫报错
        news_full.loc['全文_hot'] = '_ERROR_'
        print(news_full.loc['热搜'])
        i = not i
        time.sleep(10)
    news_hot_full = pd.concat([news_hot_full,news_full],axis=1)

  0%|          | 10/7290 [00:07<1:30:21,  1.34it/s]


In [8]:
news_hot_full.T

,上榜时间,热搜,最高排名,在榜时长（分钟）,热度,备注,交易日,时间,全文,全文_hot
55,2022-07-01 00:22:34,祝福香港祝福祖国,1,11时9分,123114,None,2022-07-01,preopen,'【#一人一句祝福香港祝福祖国#】奋进25载，起航新征程，香港未来必将在与祖国携手共进中不断...,'【#香港始终和祖国紧紧相依#】1997年香港回归，市民纷纷走上街头庆祝，“个个都兴奋得不得...
147,2022-07-01 10:05:49,1997年7月1日0时0分0秒,1,12时2分,119871,None,2022-07-01,opening,'【今天，重温#1997年7月1日0时0分0秒#，依旧泪目！】1997年7月1日0时0分0秒...,'【今天，重温#1997年7月1日0时0分0秒#，依旧泪目！】1997年7月1日0时0分0秒...
88,2022-07-01 07:15:37,东方之珠,1,11时1分,118003,None,2022-07-01,preopen,'【#驻港部队各营区举行升旗仪式#，五星红旗在香港高高飘扬】25年前的今天，香港回归祖国，中...,'【#驻港部队各营区举行升旗仪式#，五星红旗在香港高高飘扬】25年前的今天，香港回归祖国，中...
21,2022-07-01 00:00:10,钟薛高,1,13时23分,86009,None,2022-07-01,preopen,'#钟薛高31度室温下放1小时不化#？【钟薛高31℃放1小时不化？客服：融化呈粘稠状，有少量...,'#钟薛高31度室温下放1小时不化#？【钟薛高31℃放1小时不化？客服：融化呈粘稠状，有少量...
54,2022-07-01 00:21:32,七月,1,9时8分,160429,None,2022-07-01,preopen,'#MiniBAZAAR七月刊##任嘉伦雨后情天芭莎大片#每个角色都独一无二，每一段旅途都是...,'#MiniBAZAAR七月刊##任嘉伦雨后情天芭莎大片#每个角色都独一无二，每一段旅途都是...
77,2022-07-01 04:39:20,河北健康码有大变化,2,5时19分,142512,None,2022-07-01,preopen,'【#河北健康码乘车功能试点上线#】今天（6月30日）上午，河北健康码乘车功能上线启动仪式在...,'#河北健康码有大变化#河北为什么一直很穷，这几年疫情越来越穷，不是没有道理的……'
52,2022-07-01 00:07:38,全红婵陈芋汐女双10米台夺金,2,7时6分,39349,None,2022-07-01,preopen,'🥇#全红婵陈芋汐女双10米台夺金#陈芋汐@陈乐乐Diving和@全红婵势不可挡，以368....,'今日跳水领奖图来啦📸️恭喜🇨🇳@全红婵@陈乐乐Diving@Diving王宗源！#全红婵陈...
95,2022-07-01 07:28:30,杜兰特申请交易,2,5时28分,109278,None,2022-07-01,preopen,'菲尼克斯节目主持人FlexFromJersey：消息源透露，太阳、篮网、爵士正在严肃讨论、...,'刚醒有点懵，阿杜这个…是…字面意思吗#2022NBA自由市场##杜兰特申请交易#'
131,2022-07-01 09:09:46,香港纪律部队全面转用中式步操,2,5时55分,72958,None,2022-07-01,preopen,'【#香港纪律部队全面转用中式步操#】7月1日早上8点，香港特区政府在金紫荆广场，举行了升旗...,'#香港纪律部队全面转用中式步操#香港中式步操一小步，去殖民化一大步！！！'
179,2022-07-01 11:38:44,邓小平和撒切尔夫人谈判珍贵原声,2,5时7分,101444,None,2022-07-01,opening,'【#邓小平和撒切尔夫人谈判珍贵原声#“主权问题是不能够谈判的”】#香港和平回归过程有多艰难...,'【#邓小平和撒切尔夫人谈判珍贵原声#“主权问题是不能够谈判的”】#香港和平回归过程有多艰难...
